In [41]:
import torch
import nbimporter
import numpy as np
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import DataLoader

from two_sats import SatelliteDataset, ConvNet
from sentinal_1 import Sentinel1Dataset, Sentinel1ConvNet
from sentinal_2 import Sentinel2Dataset, Sentinel2ConvNet

In [42]:
# Load the .h5 file into memory once

h5_file_path_test = r"C:\Users\nadav.k\Documents\DS\DL_classification\classification_data\testing.h5"

# Open the H5 files
h5_test = h5py.File(h5_file_path_test, 'r')
test_sen1_data = h5_test['sen1']
test_sen2_data = h5_test['sen2']
test_labels = h5_test['label']


In [43]:
from torch.utils.data import DataLoader

def get_dataloader(dataset_type, batch_size=64, shuffle=False):
    """
    Returns a DataLoader for the specified dataset type.

    Args:
        dataset_type (str): Type of the dataset ('SatelliteDataset', 'Sentinel1Dataset', 'Sentinel2Dataset').
        batch_size (int): Batch size for the DataLoader.
        shuffle (bool): Whether to shuffle the dataset.

    Returns:
        DataLoader: DataLoader for the specified dataset.
    """
    if dataset_type == "SatelliteDataset":
        dataset = SatelliteDataset(sen1_data=test_sen1_data, sen2_data=test_sen2_data, labels=test_labels)
    elif dataset_type == "Sentinel1Dataset":
        dataset = Sentinel1Dataset(sen1_data=test_sen1_data, labels=test_labels)
    elif dataset_type == "Sentinel2Dataset":
        dataset = Sentinel2Dataset(sen2_data=test_sen2_data, labels=test_labels)
    else:
        raise ValueError(f"Unsupported dataset type: {dataset_type}")

    # Create and return the DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    print(f"{dataset_type} loaded with {len(dataset)} samples.")
    return dataloader


In [44]:
# General function to load a model
def load_model(model_class, path, num_classes=17, device="cuda"):
    """
    Load a saved model from the specified path.

    Args:
        model_class: The class of the model to instantiate.
        path: Full path to the saved model (e.g., './models/sentinel2_classification_model.pth').
        num_classes: Number of classes for the model.
        device: Device to load the model ('cuda' or 'cpu').

    Returns:
        The loaded model.
    """
    model = model_class(num_classes=num_classes)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    model.eval()
    print(f"Model loaded from {path}")
    return model


In [45]:
def analyze_model_performance_general(model, test_loader, num_classes=17, satellite_type="both", device="cuda"):
    """
    General evaluation function for Sentinel-1, Sentinel-2, or both.

    Args:
        model: The trained model to evaluate.
        test_loader: DataLoader for the test dataset.
        num_classes: Number of classes.
        satellite_type: 'sentinel1', 'sentinel2', or 'both'.
        device: Device for computation ('cuda' or 'cpu').
    """
    model.to(device)
    model.eval()

    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)

            # Get predicted labels
            preds = torch.argmax(outputs, dim=1)

            # Append to lists
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(preds.cpu().numpy())

    # Generate confusion matrix
    true_labels = np.array(true_labels)
    predicted_labels = np.array(predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels, labels=range(num_classes))

    # Display confusion matrix
    print(f"Confusion Matrix for {satellite_type.capitalize()}:\n", cm)
    ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(num_classes)).plot(cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix - {satellite_type.capitalize()}")
    plt.show()


        # Correct vs incorrect predictions
    correct_per_label = np.diag(cm)
    total_per_label = np.sum(cm, axis=1)
    incorrect_per_label = total_per_label - correct_per_label

    plt.figure(figsize=(12, 6))
    x = np.arange(len(correct_per_label))
    bar_width = 0.4

    # Plot bars
    correct_bars = plt.bar(x - bar_width / 2, correct_per_label, width=bar_width, label="Correct", color="g")
    incorrect_bars = plt.bar(x + bar_width / 2, incorrect_per_label, width=bar_width, label="Incorrect", color="r")

    # Add labels above bars
    for bar in correct_bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, f"{int(yval)}", ha="center", va="bottom")

    for bar in incorrect_bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, f"{int(yval)}", ha="center", va="bottom")

    # Formatting
    plt.xticks(ticks=x, labels=range(num_classes))
    plt.title(f"Correct vs Incorrect Predictions - {satellite_type.capitalize()}")
    plt.xlabel("Labels")
    plt.ylabel("Count")
    plt.legend()
    plt.grid(axis="y")
    plt.show()


In [52]:
# Define paths to saved model and test H5 file
saved_model_path = r"C:\Users\nadav.k\Documents\DS\DL_classification\saved_models\two_stas_best_manual_fulldata.pth"
h5_file_path_test = r"C:\Users\nadav.k\Documents\DS\DL_classification\classification_data\testing.h5"

# Step 1: Load the H5 test file
print("Loading test data from H5 file...")
h5_test = h5py.File(h5_file_path_test, 'r')
test_sen1_data = h5_test['sen1']
test_sen2_data = h5_test['sen2']
test_labels = h5_test['label']
print(f"Test data loaded. Samples: {len(test_labels)}")

# Step 2: Get the DataLoader
print("Creating DataLoader...")
test_loader = get_dataloader(dataset_type="SatelliteDataset", batch_size=64, shuffle=False)

# Step 3: Load the trained model
print("Loading the trained model...")
model = load_model(ConvNet, path=saved_model_path, num_classes=17, device="cuda")

# Step 4: Analyze model performance
print("Analyzing model performance...")
analyze_model_performance_general(
    model=model,
    test_loader=test_loader,
    num_classes=17,
    satellite_type="SatelliteDataset",  # Adjust this if needed
    device="cuda"
)

# Step 5: Clean up resources
h5_test.close()
print("Evaluation complete. Test H5 file closed.")


Loading test data from H5 file...
Test data loaded. Samples: 79894
Creating DataLoader...
SatelliteDataset loaded with 79894 samples.
Loading the trained model...
Model loaded from C:\Users\nadav.k\Documents\DS\DL_classification\saved_models\two_stas_best_manual_fulldata.pth
Analyzing model performance...


C:\Users\nadav.k\AppData\Local\Temp\ipykernel_9176\2026791148.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_location=device

ValueError: too many values to unpack (expected 2)

In [47]:
def analyze_model_performance_general_with_groups(model, test_loader, num_classes=17, satellite_type="both", device="cuda"):
    """
    General evaluation function for Sentinel-1, Sentinel-2, or both, with grouped bar chart for label categories.

    Args:
        model: The trained model to evaluate.
        test_loader: DataLoader for the test dataset.
        num_classes: Number of classes.
        satellite_type: 'sentinel1', 'sentinel2', or 'both'.
        device: Device for computation ('cuda' or 'cpu').
    """
    model.to(device)
    model.eval()

    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)

            # Get predicted labels
            preds = torch.argmax(outputs, dim=1)

            # Append to lists
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(preds.cpu().numpy())

    # Generate confusion matrix
    true_labels = np.array(true_labels)
    predicted_labels = np.array(predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels, labels=range(num_classes))

    # Display confusion matrix
    print(f"Confusion Matrix for {satellite_type.capitalize()}:\n", cm)
    ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(num_classes)).plot(cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix - {satellite_type.capitalize()}")
    plt.show()

    # Correct vs incorrect predictions
    correct_per_label = np.diag(cm)
    total_per_label = np.sum(cm, axis=1)
    incorrect_per_label = total_per_label - correct_per_label

    # Standard bar chart
    plt.figure(figsize=(12, 6))
    x = np.arange(len(correct_per_label))
    bar_width = 0.4

    correct_bars = plt.bar(x - bar_width / 2, correct_per_label, width=bar_width, label="Correct", color="g")
    incorrect_bars = plt.bar(x + bar_width / 2, incorrect_per_label, width=bar_width, label="Incorrect", color="r")

    for bar in correct_bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, f"{int(yval)}", ha="center", va="bottom")

    for bar in incorrect_bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, f"{int(yval)}", ha="center", va="bottom")

    plt.xticks(ticks=x, labels=range(num_classes))
    plt.title(f"Correct vs Incorrect Predictions - {satellite_type.capitalize()}")
    plt.xlabel("Labels")
    plt.ylabel("Count")
    plt.legend()
    plt.grid(axis="y")
    plt.show()

    # Grouped categories
    label_groups = {
        "Compact Urban": [0, 1, 2],
        "Light Urban": [3, 4, 5, 8],
        "Lightweight Building": [6],
        "Industrial": [7, 9],
        "Forested": [10, 11],
        "Low Plants": [12, 13],
        "Bare Rock/Paved": [14],
        "Bare Soil/Sand": [15],
        "Water": [16]
    }

    grouped_correct = []
    grouped_incorrect = []
    group_labels = []

    for group_name, indices in label_groups.items():
        grouped_correct.append(np.sum([correct_per_label[i] for i in indices]))
        grouped_incorrect.append(np.sum([incorrect_per_label[i] for i in indices]))
        group_labels.append(group_name)

    # Grouped bar chart
    plt.figure(figsize=(12, 6))
    x_grouped = np.arange(len(grouped_correct))
    correct_bars_grouped = plt.bar(x_grouped - bar_width / 2, grouped_correct, width=bar_width, label="Correct", color="g")
    incorrect_bars_grouped = plt.bar(x_grouped + bar_width / 2, grouped_incorrect, width=bar_width, label="Incorrect", color="r")

    for bar in correct_bars_grouped:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, f"{int(yval)}", ha="center", va="bottom")

    for bar in incorrect_bars_grouped:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, f"{int(yval)}", ha="center", va="bottom")

    plt.xticks(ticks=x_grouped, labels=group_labels, rotation=45, ha="right")
    plt.title(f"Grouped Correct vs Incorrect Predictions - {satellite_type.capitalize()}")
    plt.xlabel("Groups")
    plt.ylabel("Count")
    plt.legend()
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()


In [48]:
test_loader = get_dataloader(dataset_type="Sentinel1Dataset", batch_size=64, shuffle=False)


Sentinel1Dataset loaded with 79894 samples.


In [49]:
analyze_model_performance_general_with_groups(
    model=model,
    test_loader=test_loader,
    num_classes=17,
    satellite_type="sentinel2",  # 'sentinel1', 'sentinel2', or 'both'
    device="cuda"
)


NameError: name 'model' is not defined